In [1]:
import csv
import re
import hashlib
import requests


NEWS_API_KEY = '8587fb43b56040e7b46da2ed52283f57'
CSV_FILE = 'regno.csv'


def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email)


def is_valid_password(password):
    if len(password) >= 8 and any(char in '!@#$%^&*()_+' for char in password):
        return True
    return False


def signup():
    try:
        email = input("Enter your email: ")
        if not is_valid_email(email):
            print("Invalid email format.")
            return

        password = input("Enter your password (min 8 chars, 1 special char): ")
        if not is_valid_password(password):
            print("Password doesn't meet the requirements.")
            return

        security_question = input("What is your favorite color (for password recovery)?: ")

        hashed_password = hash_password(password)

        with open(CSV_FILE, 'a', newline='') as file:

            file.write(f"{email},{hashed_password},{security_question}\n")
        print("Signup successful! Please login.")
    except Exception as e:
        print(f"An error occurred during signup: {e}")


def login():
    attempts = 0
    try:
        while attempts < 5:
            email = input("Enter your email: ")
            password = input("Enter your password: ")
            hashed_password = hash_password(password)

            if not is_valid_email(email):
                print("Invalid email format.")
                attempts += 1
                continue

            with open(CSV_FILE, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    if row[0] == email and row[1] == hashed_password:
                        print("Login successful!")
                        fetch_news()
                        return
                print("Invalid credentials.")
                attempts += 1

        print("Too many failed attempts. Try again later.")
    except FileNotFoundError:
        print(f"Error: {CSV_FILE} not found.")
    except Exception as e:
        print(f"An error occurred during login: {e}")


def forgot_password():
    try:
        email = input("Enter your registered email: ")
        with open(CSV_FILE, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[0] == email:
                    security_answer = input(f"Security question - Enter your Favourite color?: ")
                    if security_answer == row[2]:
                        new_password = input("Enter your new password: ")
                        if is_valid_password(new_password):
                            update_password(email, new_password)
                            print("Password reset successful.")
                        else:
                            print("Password doesn't meet the requirements.")
                    else:
                        print("Security answer incorrect.")
                    return
        print("Email not found.")
    except FileNotFoundError:
        print(f"Error: {CSV_FILE} not found.")
    except Exception as e:
        print(f"An error occurred during password reset: {e}")


def update_password(email, new_password):
    try:
        rows = []
        hashed_password = hash_password(new_password)
        with open(CSV_FILE, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[0] == email:
                    row[1] = hashed_password
                rows.append(row)

        with open(CSV_FILE, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rows)
            
    except FileNotFoundError:
        print(f"Error: {CSV_FILE} not found.")
    except Exception as e:
        print(f"An error occurred while updating the password: {e}")


def fetch_news():
    try:
        keyword = input("Enter a keyword to search for news: ")
        url = f'https://newsapi.org/v2/everything?q={keyword}&apiKey={NEWS_API_KEY}'
        response = requests.get(url)

        if response.status_code == 200:
            articles = response.json().get('articles', [])[:5]
            if articles:
                print(f"Top 5 news articles for '{keyword}':")
                for i, article in enumerate(articles, start=1):
                    print(f"{i}. {article['title']} (Source: {article['source']['name']})")
            else:
                print("No news articles found for this keyword.")
        else:
            print("Failed to fetch news. Please check your API key or internet connection.")
            
    except requests.exceptions.RequestException as e:
        print(f"Network error occurred: {e}")
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")


def main():
    print("Welcome to the News App")
    while True:
        print("\n1. Signup\n2. Login\n3. Forgot Password\n4. Exit")
        try:
            choice = input("Choose an option: ")

            if choice == '1':
                signup()
            elif choice == '2':
                login()
            elif choice == '3':
                forgot_password()
            elif choice == '4':
                print("Goodbye!")
                break
            else:
                print("Invalid option. Please try again.")
        except Exception as e:
            print(f"An error occurred in the main menu: {e}")

if __name__ == "__main__":
    main()


Welcome to the News App

1. Signup
2. Login
3. Forgot Password
4. Exit


Choose an option:  akash@gmail.com


Invalid option. Please try again.

1. Signup
2. Login
3. Forgot Password
4. Exit


Choose an option:  1
Enter your email:  akash@gmail.com
Enter your password (min 8 chars, 1 special char):  Akash@123
What is your favorite color (for password recovery)?:  red


Signup successful! Please login.

1. Signup
2. Login
3. Forgot Password
4. Exit


...

Login successful!


Enter a keyword to search for news:  mobile


Top 5 news articles for 'mobile':
1. [Removed] (Source: [Removed])
2. [Removed] (Source: [Removed])
3. Would You Vote From Your Phone? (Source: Wired)
4. Verizon confirms a network outage is affecting mobile customers across the US (Source: Yahoo Entertainment)
5. Destiny: Rising is Bungie’s new mobile game published by NetEase Games (Source: Yahoo Entertainment)

1. Signup
2. Login
3. Forgot Password
4. Exit


Choose an option:  4


Goodbye!
